# Divergence RSI Trap - Detection(Predict, Classification)

### Import Library

In [33]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [34]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [35]:
data = dataset.copy()

In [36]:
data = data[data.index > '2020-11-01 00:00:00']

In [37]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207


In [38]:
data['RSI'] = ta.rsi(data["Close"], length=14)

In [39]:
# Identify Higher High (HH) and Lower RSI
data['HighestHigh'] = data['High'].shift(1).rolling(20).max()
data['HH'] = data['High'] > data['HighestHigh'] + 0.1
data['Lower_RSI'] = data['RSI'] < data['RSI'].shift(5).rolling(20).max()
# Identify Lower Low (LL) and Higher RSI
data['LowestLow'] = data['Low'].shift(1).rolling(20).min()
data['LL'] = data['Low'] < data['LowestLow'] - 0.1
data['Higher_RSI'] = data['RSI'] > data['RSI'].shift(5).rolling(20).min()

# Find divergence signals
data['Bullish_Divergence'] = data['LL'] & data['Higher_RSI']
# Find divergence signals
data['Bearish_Divergence'] = data['HH'] & data['Lower_RSI']

In [40]:
data[(data.Bullish_Divergence == True) | (data.Bearish_Divergence == True)]

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,LowestLow,LL,Higher_RSI,Bullish_Divergence,Bearish_Divergence
Date,,,,,,,,,,,,,,
2020-11-03 10:30:00,908.5,909.7,908.5,909.0,3935,63.300181,909.4,True,True,904.0,False,True,False,True
2020-11-03 14:15:00,906.7,910.4,906.2,909.7,5975,63.071441,909.9,True,True,903.7,False,True,False,True
2020-11-04 11:05:00,914.0,914.6,913.3,913.3,2317,64.891375,914.3,True,True,906.7,False,True,False,True
2020-11-04 13:55:00,912.8,917.2,912.5,916.3,5238,66.296297,914.6,True,True,911.8,False,True,False,True
2020-11-04 14:00:00,916.6,918.1,916.4,916.7,5942,67.446991,917.2,True,True,911.8,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13 10:15:00,1327.9,1328.1,1326.5,1327.0,5126,24.576424,1337.6,False,True,1327.4,True,True,True,False
2025-02-13 10:20:00,1326.9,1327.5,1326.2,1326.4,2910,23.121278,1337.1,False,True,1326.5,True,True,True,False
2025-02-13 13:45:00,1337.2,1338.5,1336.5,1336.9,4460,66.257459,1338.3,True,True,1328.6,False,True,False,True


## Labeling for trap

In [41]:
traps = []
for i, row in data.iterrows():
    if row['Bearish_Divergence'] or row['Bullish_Divergence']:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date + ' 14:30:00')]
        if not len(data_to_end_day):
            traps.append(0)
            continue
        #
        if row['Bearish_Divergence']:
            if len(data_to_end_day[data_to_end_day.High > row['High']]) > 0:
                traps.append(1)
            else:
                traps.append(0)
        else:
            if len(data_to_end_day[data_to_end_day.Low < row['Low']]) > 0:
                traps.append(1)
            else:
                traps.append(0)
    else:
        traps.append(0)

In [42]:
data['trap'] = traps

In [43]:
data[((data.Bullish_Divergence == True) | (data.Bearish_Divergence == True)) & (data.trap == 1)]

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,LowestLow,LL,Higher_RSI,Bullish_Divergence,Bearish_Divergence,trap
Date,,,,,,,,,,,,,,,
2020-11-03 10:30:00,908.5,909.7,908.5,909.0,3935,63.300181,909.4,True,True,904.0,False,True,False,True,1
2020-11-04 11:05:00,914.0,914.6,913.3,913.3,2317,64.891375,914.3,True,True,906.7,False,True,False,True,1
2020-11-04 13:55:00,912.8,917.2,912.5,916.3,5238,66.296297,914.6,True,True,911.8,False,True,False,True,1
2020-11-05 10:20:00,915.7,917.0,915.7,916.8,2250,63.475309,916.3,True,True,910.2,False,True,False,True,1
2020-11-05 10:25:00,916.8,917.2,916.5,917.1,1749,64.729889,917.0,True,True,912.3,False,True,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-11 09:45:00,1330.7,1330.7,1329.4,1329.4,3681,39.061852,1336.2,False,True,1329.6,True,True,True,False,1
2025-02-13 09:15:00,1330.8,1331.0,1329.3,1329.7,3765,24.085128,1340.7,False,True,1329.5,True,True,True,False,1
2025-02-13 09:20:00,1329.7,1330.3,1328.5,1330.2,3555,27.482068,1340.7,False,True,1329.3,True,True,True,False,1


In [44]:
data[((data.Bullish_Divergence == True) | (data.Bearish_Divergence == True)) & (data.trap == 0)]

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,LowestLow,LL,Higher_RSI,Bullish_Divergence,Bearish_Divergence,trap
Date,,,,,,,,,,,,,,,
2020-11-03 14:15:00,906.7,910.4,906.2,909.7,5975,63.071441,909.9,True,True,903.7,False,True,False,True,0
2020-11-04 14:00:00,916.6,918.1,916.4,916.7,5942,67.446991,917.2,True,True,911.8,False,True,False,True,0
2020-11-06 13:25:00,905.8,906.5,903.3,904.3,5390,27.693535,911.8,False,True,905.0,True,True,True,False,0
2020-11-09 14:20:00,921.0,922.2,920.9,921.2,6445,66.567163,921.3,True,True,916.0,False,True,False,True,0
2020-11-11 14:15:00,912.6,914.1,912.2,913.5,7909,33.093142,922.8,False,True,912.6,True,True,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-12 13:30:00,1339.9,1340.7,1339.8,1339.8,3092,58.392608,1340.2,True,True,1338.3,False,True,False,True,0
2025-02-13 10:20:00,1326.9,1327.5,1326.2,1326.4,2910,23.121278,1337.1,False,True,1326.5,True,True,True,False,0
2025-02-13 13:45:00,1337.2,1338.5,1336.5,1336.9,4460,66.257459,1338.3,True,True,1328.6,False,True,False,True,0
